In [1]:
from ydata_profiling import ProfileReport
import pandas as pd

raw_data = pd.read_csv('/home/templar/Desktop/projects/zaz/client-projects/ai-bobby-classification/data/raw/gelation_hardness_oct_2024.csv')

In [2]:
#profile = ProfileReport(raw_data, title="Raw Data Report", explorative=True)
#profile.to_file('../data/report/raw_data_report.html')

## Observations from the Raw data report

### Per column comments

* There are 586 rows, from 120 different articles in the dataset.
* 43 Distinct types of proteins, soy protein is the most common with 101 observations, followed by Whey protein with 77 observations.
* There are a lot of protein types that are not represented in the dataset. This is something to be aware of.
* The most common type of treatment is 'no special treatment', with 293 observations.
* Treatment temperature is missing for 364 observations.
* Treatment time is missing for 297 observations.
* The most common type of additive is 'no additives', with 246 observations.
* pH is missing for 125 observations (21,3%).
* The most common type of salt is 'no salt', with 336 observations (57,3%).
* Ionic strength most common value is 0, with 343 observations (58,5%).
* Heating temperature (°C) for gel preparation has 71 (12.1%) missing values.
* Heating/hold time (min) also has 71 (12.1%) missing values.
* Storage time (h) has 191 missing values (32,6%).
* There are 538 gel formation (91,8%) observations, and 48 non-gel formation observations (8,2%).

### General comments
* Need to define strategy for handling missing values.
* Need to define strategy for handling protein types that are not represented in the dataset.
* Treatment condition code, treatment condition value, treatment temperature, storage time and treatment time have a lot of missing values. Can we drop those columns or are they crucial?
* Does the unit of measure changes for any of the columns?
* Suggestions on data quality checks?




## Convert data types and drop unnecessary columns

In [3]:
# Get data
df = raw_data.copy()

# Get only rows with gel formation
df = df[df['If a gel can be formed (0-1)'] == 1]


# Define categorical columns
categorical_columns = ['Protein codes', 'Type of salt', 'Additives', 'Treatment code']

# Define numeric columns
numerical_columns = [
    'Samples stored (°C)',
    'ionic strength (M)',
    'Additives Concentration (%)',
    'Protein Concentration (%)',
    'pH',
    'Heating temperature (°C) for gel preparation',
    'Heating/hold time (min)',
    'Hardness/firmness/strength (g)'
]

# Convert object columns to categorical
for col in categorical_columns:
    df[col] = df[col].astype('category')

# Convert columns to numeric
for col in numerical_columns:
    df[col] = df[col].astype('float64')


In [7]:
# Columns to drop
columns_to_drop = [
    'Citation',
    'Citation Link',
    'Protein',
    'Treatment condition code',
    'Treatment condition value',
    'Treatment temperature ( °C)',
    'Treatment time (min)',
    'Storage time (h)',
    'If a gel can be formed (0-1)',
]

# Drop columns
df_clean = df.drop(columns=columns_to_drop, axis=1)

# Drop rows with missing target values
df_clean = df_clean.dropna(subset=['Hardness/firmness/strength (g)'])

## Generate clean data report

In [10]:
profile = ProfileReport(df_clean, title="Clean Data Report", explorative=True)
profile.to_file('../data/report/clean_data_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Train first model

In [14]:
# Define features and target
X = df_clean.drop('Hardness/firmness/strength (g)', axis=1).values
y = df_clean['Hardness/firmness/strength (g)'].values

# Split the data
# from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import KFold
import numpy as np

def train_and_evaluate_model(X, y, k=5, model=lgb.LGBMRegressor(), metrics_list=[mean_absolute_error, mean_squared_error]):
  """
  Trains and evaluates a regression model using K-fold cross-validation.

  Args:
    X: The feature matrix.
    y: The target variable.
    k: The number of folds for cross-validation.
    model: The regression model to use.
    metrics_list: A list of metric functions to evaluate the model.

  Returns:
    A dictionary of evaluation results for each fold and the average across all folds.
  """

  kf = KFold(n_splits=k, shuffle=True, random_state=42)
  results = []

  for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)


    fold_results = {}
    for metric in metrics_list:
      metric_name = metric.__name__
      fold_results[metric_name] = metric(y_test, y_pred)
    results.append(fold_results)

  # Calculate average results across all folds
  avg_results = {}
  for metric_name in results[0].keys():
    avg_results[metric_name] = np.mean([fold[metric_name] for fold in results])

  return {'folds': results, 'average': avg_results}

In [15]:
results = train_and_evaluate_model(
    X, y, k=5, model=lgb.LGBMRegressor(), 
    metrics_list=[r2_score, mean_absolute_error, root_mean_squared_error, mean_squared_error, mean_absolute_percentage_error]
)

print(results)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000339 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 222
[LightGBM] [Info] Number of data points in the train set: 426, number of used features: 11
[LightGBM] [Info] Start training from score 2852.267812
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [17]:
from pprint import pprint
pprint(results['average'])


{'mean_absolute_error': np.float64(2223.2047714006344),
 'mean_absolute_percentage_error': np.float64(8.124609880353712e+16),
 'mean_squared_error': np.float64(68648119.74223295),
 'r2_score': np.float64(0.6763551775070873),
 'root_mean_squared_error': np.float64(7072.748413567654)}


## Traininng a wider set of models

In [25]:
from sklearn.pipeline import Pipeline

def train_and_evaluate_model(X, y, k=5, model=Pipeline(steps=[]), 
                            metrics_list=[mean_absolute_error, mean_squared_error], 
                            numerical_cols=None, categorical_cols=None):
  """
  Trains and evaluates a regression model using K-fold cross-validation.

  Args:
    X: The feature matrix.
    y: The target variable.
    k: The number of folds for cross-validation.
    model: The regression model to use (must be a Pipeline if preprocessing is needed).
    metrics_list: A list of metric functions to evaluate the model.
    numerical_cols: List of numerical column names (needed for preprocessing).
    categorical_cols: List of categorical column names (needed for preprocessing).

  Returns:
    A dictionary of evaluation results for each fold and the average across all folds.
  """

  kf = KFold(n_splits=k, shuffle=True, random_state=42)
  results = []

  for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train_df = pd.DataFrame(X_train, columns=numerical_cols.tolist() + categorical_cols.tolist())
    X_test_df = pd.DataFrame(X_test, columns=numerical_cols.tolist() + categorical_cols.tolist())
    model.fit(X_train_df, y_train)  # Fit with DataFrame
    y_pred = model.predict(X_test_df)  # Predict with DataFrame

    fold_results = {}
    for metric in metrics_list:
      metric_name = metric.__name__
      fold_results[metric_name] = metric(y_test, y_pred)
    results.append(fold_results)

  # Calculate average results across all folds
  avg_results = {}
  for metric_name in results[0].keys():
    avg_results[metric_name] = np.mean([fold[metric_name] for fold in results])

  return avg_results

In [31]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from tqdm import tqdm
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Create transformers for numerical and categorical features
numerical_transformer = SimpleImputer(strategy='median')
categorical_transformer = SimpleImputer(strategy='most_frequent')

X = df_clean.drop('Hardness/firmness/strength (g)', axis=1)
y = df_clean['Hardness/firmness/strength (g)']

numerical_cols = X.select_dtypes(include=np.number).columns
categorical_cols = X.select_dtypes(include=['category']).columns

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
RANDOM_STATE = 42

models = {
    "Linear Regression": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    "Ridge": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', Ridge(random_state=RANDOM_STATE))]),
    "Lasso": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', Lasso(random_state=RANDOM_STATE))]),
    "ElasticNet": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', ElasticNet(random_state=RANDOM_STATE))]),
    "Decision Tree": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', DecisionTreeRegressor(random_state=RANDOM_STATE))]),
    "Random Forest": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(random_state=RANDOM_STATE))]),
    "Gradient Boosting": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(random_state=RANDOM_STATE))]),
    "LightGBM": Pipeline(steps=[('regressor', lgb.LGBMRegressor(random_state=RANDOM_STATE, verbose=-1, verbose_eval=False))])
}

all_results = {}

for model_name, model in tqdm(models.items()):
    print(f"Training {model_name}...")
    avg_results = train_and_evaluate_model(
        X.values, y.values, model=model, 
        numerical_cols=numerical_cols, 
        categorical_cols=categorical_cols
    )
    all_results[model_name] = avg_results

# Print the results
for model_name, results in all_results.items():
    print(f"\n{model_name} Results:")
    for metric_name, value in results.items():
        print(f"  {metric_name}: {value:.4f}")

# You can further process the all_results dictionary, 
# e.g., convert it to a pandas DataFrame for easier analysis and comparison.
results_df = pd.DataFrame(all_results).T

 50%|█████     | 4/8 [00:00<00:00, 34.48it/s]

Training Linear Regression...
Training Ridge...
Training Lasso...
Training ElasticNet...
Training Decision Tree...
Training Random Forest...
Training Gradient Boosting...
Training LightGBM...


100%|██████████| 8/8 [00:01<00:00,  6.27it/s]


Linear Regression Results:
  mean_absolute_error: 7505.0229
  mean_squared_error: 240795825.7878

Ridge Results:
  mean_absolute_error: 7230.6356
  mean_squared_error: 240940438.9401

Lasso Results:
  mean_absolute_error: 7502.9880
  mean_squared_error: 240812832.0220

ElasticNet Results:
  mean_absolute_error: 6602.1156
  mean_squared_error: 348034001.0029

Decision Tree Results:
  mean_absolute_error: 584.0742
  mean_squared_error: 4375193.5627

Random Forest Results:
  mean_absolute_error: 567.3758
  mean_squared_error: 3112755.0970

Gradient Boosting Results:
  mean_absolute_error: 711.7878
  mean_squared_error: 3864970.5711

LightGBM Results:
  mean_absolute_error: 2223.2048
  mean_squared_error: 68648119.7422


In [34]:
# print sorted by mean absolute error
print("\nSummary of Results:")
print(results_df.sort_values(by='mean_absolute_error', ascending=True))


Summary of Results:
                   mean_absolute_error  mean_squared_error
Random Forest               567.375805        3.112755e+06
Decision Tree               584.074192        4.375194e+06
Gradient Boosting           711.787763        3.864971e+06
LightGBM                   2223.204771        6.864812e+07
ElasticNet                 6602.115558        3.480340e+08
Ridge                      7230.635572        2.409404e+08
Lasso                      7502.988006        2.408128e+08
Linear Regression          7505.022945        2.407958e+08


### Using scaler

In [36]:
from sklearn.preprocessing import StandardScaler

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = SimpleImputer(strategy='most_frequent')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
    ])

RANDOM_STATE = 42

models = {
    "Linear Regression": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    "Ridge": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', Ridge(random_state=RANDOM_STATE))]),
    "Lasso": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', Lasso(random_state=RANDOM_STATE))]),
    "ElasticNet": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', ElasticNet(random_state=RANDOM_STATE))]),
    "Decision Tree": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', DecisionTreeRegressor(random_state=RANDOM_STATE))]),
    "Random Forest": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(random_state=RANDOM_STATE))]),
    "Gradient Boosting": Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(random_state=RANDOM_STATE))]),
    "LightGBM": Pipeline(steps=[('regressor', lgb.LGBMRegressor(random_state=RANDOM_STATE, verbose=-1, verbose_eval=False))])
}

scaler_results = {}

for model_name, model in tqdm(models.items()):
    print(f"Training {model_name}...")
    avg_results = train_and_evaluate_model(
        X.values, y.values, model=model, 
        numerical_cols=numerical_cols, 
        categorical_cols=categorical_cols,
        metrics_list=[r2_score, mean_absolute_error, root_mean_squared_error, mean_squared_error, mean_absolute_percentage_error]
    )
    scaler_results[model_name] = avg_results

# Print the results
for model_name, results in scaler_results.items():
    print(f"\n{model_name} Results:")
    for metric_name, value in results.items():
        print(f"  {metric_name}: {value:.4f}")

# You can further process the all_results dictionary, 
# e.g., convert it to a pandas DataFrame for easier analysis and comparison.
scaler_results_df = pd.DataFrame(scaler_results).T
print(scaler_results_df.sort_values(by='mean_absolute_error', ascending=True))

 38%|███▊      | 3/8 [00:00<00:00, 26.55it/s]

Training Linear Regression...
Training Ridge...
Training Lasso...
Training ElasticNet...
Training Decision Tree...
Training Random Forest...


 75%|███████▌  | 6/8 [00:00<00:00,  7.10it/s]

Training Gradient Boosting...
Training LightGBM...


100%|██████████| 8/8 [00:01<00:00,  5.91it/s]


Linear Regression Results:
  r2_score: -3.6336
  mean_absolute_error: 7505.0229
  root_mean_squared_error: 15209.2413
  mean_squared_error: 240795825.7878
  mean_absolute_percentage_error: 193755591175951392.0000

Ridge Results:
  r2_score: -3.1876
  mean_absolute_error: 7223.4662
  root_mean_squared_error: 15113.5243
  mean_squared_error: 240948754.9699
  mean_absolute_percentage_error: 186225219587862912.0000

Lasso Results:
  r2_score: -3.6308
  mean_absolute_error: 7501.9108
  root_mean_squared_error: 15209.1557
  mean_squared_error: 240815765.2135
  mean_absolute_percentage_error: 193721148471638592.0000

ElasticNet Results:
  r2_score: -0.2628
  mean_absolute_error: 5339.5304
  root_mean_squared_error: 16002.4176
  mean_squared_error: 347663784.9017
  mean_absolute_percentage_error: 93057606387900000.0000

Decision Tree Results:
  r2_score: 0.8227
  mean_absolute_error: 585.8983
  root_mean_squared_error: 2062.7246
  mean_squared_error: 4378083.5554
  mean_absolute_percentage_er